<a href="https://colab.research.google.com/github/Shashank-Holla/TSAI-END-Program/blob/main/04-%20Pytorch/Session4_SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Analysis

Objective of this assignment is to train a bidirectional LSTM to predict movie sentiment.

### Import necessary packages

In [1]:
import torch
from torchtext import data, datasets
import random
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary

In [2]:
print(torch.__version__)

1.7.0+cu101


In [3]:
SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

### Prepare dataset

In [4]:
TEXT = data.Field(tokenize= 'spacy', include_lengths=True)
LABEL = data.LabelField(dtype=torch.float)

In [5]:
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

In [6]:
train_data, valid_data = train_data.split(random_state=random.seed(SEED))

In [7]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


In [8]:
# Example - text, label
# vars() function returns the __dict__ attribute of the given object.
# Dictionary returned {text:[], label:''}
print(vars(train_data.examples[0]))

{'text': ['So', 'you', 'might', 'be', 'reading', 'some', 'of', 'the', 'comments', 'posted', 'on', 'this', 'film', ',', 'and', 'you', 'might', 'be', 'thinking', 'to', 'yourself', ',', '"', 'Huh', '.', 'There', 'were', 'sure', 'a', 'bunch', 'of', 'RAVE', 'REVIEWS', 'posted', 'on', 'December', '30', '.', '"', 'Funny', 'thing', 'is', ',', 'most', 'of', 'these', 'rave', 'reviews', 'sound', 'like', 'they', "'re", 'coming', 'from', 'the', 'same', 'person', ',', 'or', 'at', 'best', 'a', 'small', ',', 'coordinated', 'group', 'of', '"', 'Open', 'House', '"', 'groupies', '.', 'The', 'truth', ',', 'my', 'friends', ',', 'is', 'that', 'this', 'film', 'is', 'truly', 'unwatchable', '.', 'Just', 'because', 'it', "'s", '"', 'independent', '"', 'does', "n't", 'mean', 'it', 'gets', 'a', 'free', 'pass', '.', 'If', 'you', "'re", 'going', 'to', 'make', 'a', 'musical', ',', 'whether', 'on', 'film', 'or', 'on', 'stage', ',', 'whether', 'on', 'Broadway', 'or', 'at', 'the', 'local', 'community', 'playhouse', ','

### Prepare reversed text for training data.

In [9]:
for i in range(len(train_data)):
  vars(train_data.examples[i]).get('text').reverse()

In [10]:
print(vars(train_data.examples[0]))

{'text': ['.', 'involved', 'everyone', 'for', 'embarrassed', 'deeply', 'felt', 'actually', 'I', 'where', 'experiences', 'going', '-', 'movie', 'unfortunate', 'those', 'of', 'one', "'s", 'It', '.', 'one', 'this', 'save', "n't", 'ca', 'Rapp', 'Anthony', 'Even', '.', 'charming', 'and', 'witty', 'are', 'that', 'lyrics', 'have', 'you', ')', 'e.', '(', 'and', ',', 'score', 'written', '-', 'well', 'a', 'have', 'you', ')', 'd.', '(', ',', 'equipment', 'sound', 'decent', 'have', 'you', ')', 'c.', '(', ',', 'dance', 'can', 'actors', 'your', ')', 'b.', '(', ',', 'sing', 'can', 'actors', 'your', ')', 'a.', '(', 'that', 'sure', 'make', 'probably', 'should', 'you', ',', 'playhouse', 'community', 'local', 'the', 'at', 'or', 'Broadway', 'on', 'whether', ',', 'stage', 'on', 'or', 'film', 'on', 'whether', ',', 'musical', 'a', 'make', 'to', 'going', "'re", 'you', 'If', '.', 'pass', 'free', 'a', 'gets', 'it', 'mean', "n't", 'does', '"', 'independent', '"', "'s", 'it', 'because', 'Just', '.', 'unwatchable'

### Use pre-trained word embeddings

In [11]:
MAX_VOCAB_SIZE = 25_000
TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE, vectors = "glove.6B.100d", unk_init = torch.Tensor.normal_)
LABEL.build_vocab(train_data)

In [12]:
BATCH_SIZE = 64

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Is Cuda available? :", torch.cuda.is_available())

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device
)

Is Cuda available? : True


### Model build

In [13]:
class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout, pad_idx):
        super(RNN, self).__init__()
        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim, padding_idx=pad_idx)
        # self.rnn = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, num_layers=n_layers, dropout=dropout, bidirectional=bidirectional)
        
        # Creating stacked LSTMs by adding single layered LSTMs
        self.rnn = nn.ModuleList()
        for i in range(n_layers):
            input_size = embedding_dim if i==0 else hidden_dim*2
            self.rnn.append(nn.LSTM(input_size=input_size, hidden_size=hidden_dim, num_layers=1, bidirectional=bidirectional))


        self.fc = nn.Linear(hidden_dim*2, output_dim)
        self.dropout = nn.Dropout(dropout)
        # self.dropout_inplace = nn.Dropout(p=dropout, inplace=True)
    
    def forward(self, text, text_lengths):
        #text = [sent len, batch size]
        embedded = self.dropout(self.embedding(text))
        #embedded = [sent len, batch size, emb dim]

        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)

        # LSTM
        # packed_output, (hidden, cell) = self.rnn(packed_embedded)
        # Implementing multilayered LSTM by for loop. Dropout is applied after every LSTM layer, except the last one.
        packed_output = packed_embedded
        for i in range(len(self.rnn)):
            if i == 0:
                packed_output, (hidden, cell) = self.rnn[i](packed_output)
            else:
                packed_output, (hidden, cell) = self.rnn[i](packed_output, (hidden, cell))
            if i != len(self.rnn)-1:
                output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)
                packed_output = nn.utils.rnn.pack_padded_sequence(self.dropout(output), output_lengths)

        # Unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)
        #output = [sent len, batch size, hid dim * num directions]

        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        #hidden = [batch size, hid dim * num directions]
            
        return self.fc(hidden)


In [14]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 3
BIDIRECTIONAL = True
DROPOUT = 0.2
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX)

In [26]:
model

RNN(
  (embedding): Embedding(25002, 100, padding_idx=1)
  (rnn): ModuleList(
    (0): LSTM(100, 256, bidirectional=True)
    (1): LSTM(512, 256, bidirectional=True)
    (2): LSTM(512, 256, bidirectional=True)
  )
  (fc): Linear(in_features=512, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [15]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 6,387,817 trainable parameters


Load pre-trained embeddings to the model's embedding layer. Shape = [vocab_size, embedding_dim]

In [16]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([25002, 100])


In [17]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.3970,  0.4024,  1.0612,  ..., -0.0136, -0.3363,  0.6442],
        [-0.5197,  1.0395,  0.2092,  ..., -0.8857, -0.2294,  0.1244],
        [ 0.0057, -0.0707, -0.0804,  ..., -0.3292, -0.0130,  0.0716]])

<UNK> and <PAD> tokens are initialized using Normal distribution during vocab build. Initialize them to zero vectors.

In [18]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.3970,  0.4024,  1.0612,  ..., -0.0136, -0.3363,  0.6442],
        [-0.5197,  1.0395,  0.2092,  ..., -0.8857, -0.2294,  0.1244],
        [ 0.0057, -0.0707, -0.0804,  ..., -0.3292, -0.0130,  0.0716]])


### Train the model

In [19]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [20]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [21]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [22]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0    
    model.train()
    
    for batch in iterator:        
        optimizer.zero_grad()
        
        text, text_lengths = batch.text 
        text = text.to(device)
        # text_lengths needs to be on cpu - https://github.com/pytorch/pytorch/issues/43227
        text_lengths = text_lengths.cpu()       
        predictions = model(text, text_lengths).squeeze(1)
        
        loss = criterion(predictions, batch.label)       
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [23]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0    
    model.eval()
    
    with torch.no_grad():    
        for batch in iterator:
            text, text_lengths = batch.text
            text = text.to(device)
            text_lengths = text_lengths.cpu()
                        
            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.label)            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [24]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [25]:
N_EPOCHS = 20

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 55s
	Train Loss: 0.636 | Train Acc: 63.30%
	 Val. Loss: 0.540 |  Val. Acc: 72.15%
Epoch: 02 | Epoch Time: 0m 55s
	Train Loss: 0.600 | Train Acc: 65.02%
	 Val. Loss: 0.695 |  Val. Acc: 54.74%
Epoch: 03 | Epoch Time: 0m 55s
	Train Loss: 0.579 | Train Acc: 69.63%
	 Val. Loss: 0.586 |  Val. Acc: 72.89%
Epoch: 04 | Epoch Time: 0m 55s
	Train Loss: 0.336 | Train Acc: 85.89%
	 Val. Loss: 0.335 |  Val. Acc: 85.91%
Epoch: 05 | Epoch Time: 0m 55s
	Train Loss: 0.227 | Train Acc: 91.34%
	 Val. Loss: 0.332 |  Val. Acc: 85.91%
Epoch: 06 | Epoch Time: 0m 55s
	Train Loss: 0.179 | Train Acc: 93.58%
	 Val. Loss: 0.381 |  Val. Acc: 86.07%
Epoch: 07 | Epoch Time: 0m 55s
	Train Loss: 0.134 | Train Acc: 95.43%
	 Val. Loss: 0.360 |  Val. Acc: 87.04%
Epoch: 08 | Epoch Time: 0m 55s
	Train Loss: 0.100 | Train Acc: 96.70%
	 Val. Loss: 0.393 |  Val. Acc: 88.03%
Epoch: 09 | Epoch Time: 0m 55s
	Train Loss: 0.073 | Train Acc: 97.71%
	 Val. Loss: 0.379 |  Val. Acc: 88.26%
Epoch: 10 | Epoch T

### Training on text that is reversed.

Training text to the model is tensor of shape (168,64) containing the word id.
Batch of text is reversed before sending to model.

In [27]:
batch = next(iter(train_iterator))
text, text_lengths = batch.text

In [28]:
print(text.shape)
text

torch.Size([381, 64])


tensor([[    4,   557,     4,  ...,     4,     4,     4],
        [  410,   165,  3414,  ...,   115,    24,   622],
        [   63,    18,  5255,  ...,    20,     6,  4562],
        ...,
        [   19,  1086,  6931,  ...,     1,     1,     1],
        [22631,  1078,   592,  ...,     1,     1,     1],
        [24627,    11,    25,  ...,     1,     1,     1]], device='cuda:0')

In [29]:
torch.fliplr(text)

tensor([[    4,     4,     4,  ...,     4,   557,     4],
        [  622,    24,   115,  ...,  3414,   165,   410],
        [ 4562,     6,    20,  ...,  5255,    18,    63],
        ...,
        [    1,     1,     1,  ...,  6931,  1086,    19],
        [    1,     1,     1,  ...,   592,  1078, 22631],
        [    1,     1,     1,  ...,    25,    11, 24627]], device='cuda:0')

Epoch: 01 | Epoch Time: 0m 39s

	Train Loss: 0.668 | Train Acc: 58.62%
	 Val. Loss: 0.640 |  Val. Acc: 66.53%

Epoch: 02 | Epoch Time: 0m 40s

	Train Loss: 0.597 | Train Acc: 68.08%
	 Val. Loss: 0.512 |  Val. Acc: 76.21%

Epoch: 03 | Epoch Time: 0m 40s

	Train Loss: 0.499 | Train Acc: 76.39%
	 Val. Loss: 0.472 |  Val. Acc: 76.07%

Epoch: 04 | Epoch Time: 0m 40s

	Train Loss: 0.468 | Train Acc: 78.68%
	 Val. Loss: 0.352 |  Val. Acc: 85.42%

Epoch: 05 | Epoch Time: 0m 40s

	Train Loss: 0.343 | Train Acc: 85.99%
	 Val. Loss: 0.292 |  Val. Acc: 88.00%


With in-place dropout
RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.cuda.FloatTensor [15646, 512]], which is output 0 of CudnnRnnBackward, is at version 1; expected version 0 instead. Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).